In [ ]:
# Instalar las dependencias necesarias
!pip install requests markdownify

# Importar las bibliotecas necesarias
import requests
import json
from collections import defaultdict

# Configuración de las solicitudes
graphql_url = "https://yourdomain.com/graphql"
headers = {
    "Authorization": "Bearer YOUR API KEY",
    "Content-Type": "application/json"
}

# Primera solicitud GraphQL para obtener la lista de páginas
query_list = '''
query Request {
  pages {
    list(orderBy: PATH) {
      id
      path
      title
      contentType
    }
  }
}
'''

response = requests.post(graphql_url, headers=headers, json={"query": query_list})

# Verificar si la solicitud fue exitosa
if response.status_code != 200:
    print(f"Error en la solicitud inicial: {response.status_code}")
    print(response.text)
else:
    # Imprimir la respuesta completa para depuración
    print("Respuesta completa de la solicitud inicial:")
    print(response.text)

    pages_list = response.json().get("data", {}).get("pages", {}).get("list", [])

    # Debugging output
    print(f"Se encontraron {len(pages_list)} páginas.")

    # Crear un diccionario para la jerarquía
    hierarchy = defaultdict(lambda: {'subpages': {}})

    for page in pages_list:
        parts = page['path'].strip('/').split('/')
        current_level = hierarchy

        for part in parts:
            if part not in current_level:
                current_level[part] = {'subpages': {}}
            current_level = current_level[part]['subpages']

        current_level['_info'] = {'id': page['id'], 'title': page['title'], 'contentType': page['contentType']}

    # Función para imprimir la jerarquía con títulos
    def print_hierarchy_with_titles(level, path='/', indent=0):
        for key, value in level.items():
            if key == '_info':
                continue
            full_path = f"{path}{key}/"
            title = next((page['title'] for page in pages_list if page['path'] == full_path.strip('/')), key)
            print('  ' * indent + title)
            if '_info' in value:
                print('  ' * (indent + 1) + f"ID: {value['_info']['id']}")
                print('  ' * (indent + 1) + f"Title: {value['_info']['title']}")
                print('  ' * (indent + 1) + f"ContentType: {value['_info']['contentType']}")
            print_hierarchy_with_titles(value['subpages'], full_path, indent + 1)

    # Imprimir la jerarquía de páginas con títulos
    print_hierarchy_with_titles(hierarchy)